In [1]:
import numpy as np
from blab.array import loadmat
from blab.array import marray_from_lists
from blab.itertools import true_meshgrid
from tqdm import tqdm
from numpy import ma
import mne
from scipy import stats
from matplotlib import pyplot as plt
import gc

In [2]:
cases = ['right_im1', 'right_im2']
bgr_case = 'right_real'

res = loadmat('/Volumes/Samsung_T5/TMS_Motion_data/DAY2_TMS.mat')['res']

channels = np.array(res[0]['right_real']['label']).T[0].tolist()

coi = ['FC5', 'FC1', 'FC2', 'FC6', 'C3', 'Cz', 'C4', 'CP5', 'CP1', 'CP2', 'CP6', 'T7', 'T8']
coi_mask = np.sum([np.array(channels) == i for i in coi], 0).astype(bool)

data_arrays = []
for case in tqdm(cases):
    temp = marray_from_lists([j[case]['trial'] if j is not None else None for j in res])[..., :9500:4][..., coi_mask, :]
    data_arrays.append(temp)

data_arrays = ma.stack(data_arrays).reshape(-1, 20, 13, 2375)
data_arrays_mask = data_arrays.mask
data_arrays = data_arrays.data
data_arrays[data_arrays_mask] = np.nan
# data_arrays = data_arrays.reshape(-1, 20, 32, 9500)[..., ::4] # reduce fsample. (cases, subs, ...) -> (cases*subs, ...) for epochs creation
# data_arrays = data_arrays[:10] # only im1 and im2

100%|██████████| 2/2 [00:15<00:00,  7.73s/it]


In [3]:
montage = mne.channels.make_standard_montage('standard_1020')
info = mne.create_info(coi, 250, ch_types='eeg')
info.set_montage(montage)

sensor_adjacency, ch_names = mne.channels.find_ch_adjacency(
    info, 'eeg')

res_epochs = [mne.EpochsArray(i, info, verbose=False) for i in data_arrays]

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 13


In [4]:
freqs = np.r_[10.5:14.5:0.4]

def transform_func(x):
    temp = mne.time_frequency.tfr_morlet(x, freqs, freqs, use_fft=True, return_itc=False, n_jobs=10, average=False, verbose=False)
    temp = temp.crop(1, 9).apply_baseline((1, 5), mode='percent', verbose=False)
    temp = temp.average(dim='freqs')
    return temp

res_tfr = [transform_func(i) for i in tqdm(res_epochs)]
all_data = np.array([np.rollaxis(np.squeeze(i.data), 1, 3) for i in res_tfr]).reshape(1, 2, -1, 20, 2001, 13)[..., 1001:, :]

100%|██████████| 40/40 [00:18<00:00,  2.11it/s]


In [20]:
# np.save(all_data, ''
np.rollaxis(np.squeeze(transform_func(res_epochs[0])), 1, 3).data.shape

(20, 2001, 13)

In [7]:
np.save('../data/eeg_tfr/TMS_TIME-CH-TRIAL_DAY2_IM1-2_MOTOR-AREA_BCORR_POST.npy', all_data)